# Sistema de evaluación DBP

Ejemplo: Un grupo de 3 estudiantes con las siguientes calificaciones:

| | **E1** | **E2** | **E3** |
|---|---|---|---|
| **E1** | 18 | 19 | 16 |
| **E2** | 14 | 15 | 16 |
| **E3** | 16 | 18 | 17 |


## Paso 1: Recolectar notas

Supongamos que tenemos un grupo de $n$ estudiantes. Cada estudiante $i$ da una autoevaluación $A_i$ y recibe calificaciones $C_{ij}$ de sus compañeros $j$ (donde $j \neq i$).

In [32]:
import numpy as np

n = 4

autoevaluaciones = np.array([20, 20, 20, 20]) 


coevaluaciones = np.array([
    [19, 16, 15],  # Evaluaciones recibidas por el primer estudiante de los otros dos
    [14, 16, 10],  # Evaluaciones recibidas por el segundo estudiante
    [16, 18, 12],   # Evaluaciones recibidas por el tercer estudiante
    [20, 19, 18]   # Evaluaciones recibidas por el cuarto estudiante
])


## Paso 2: Calcular la desviación estándar

Para cada estudiante, primero calculamos la calificación promedio que recibe de sus compañeros:

$$
prom(C_i) = \frac{1}{n-1} \sum_{j \neq i} C_{ij} \\
prom(C_1) = \frac{1}{2} \times (19+16) = 17.5 \\
prom(C_2) = \frac{1}{2} \times (14+16) = 15 \\
prom(C_3) = \frac{1}{2} \times (16+18) = 17 \\
$$

Luego, calculamos la desviación estándar de las calificaciones recibidas por cada estudiante:

$$
\sigma_i = \sqrt{\frac{1}{N-1} \sum_{j \neq i}^{N} (C_{ij} - prom(C_i))^2} \\
\sigma_1 = \sqrt{\frac{1}{1} \times [(19-17.5)^2 + (16-17.5)^2]} = 2.12132 \\
\sigma_2 = \sqrt{\frac{1}{1} \times [(14-15)^2 + (16-15)^2]} = 1.41421 \\
\sigma_3 = \sqrt{\frac{1}{1} \times [(16-17)^2 + (18-17)^2]} = 1.41421 \\
$$

In [33]:
# Calcular la calificación promedio recibida por cada estudiante
promedios = np.mean(coevaluaciones, axis=1)
print(promedios)

# Calcular la desviación estándar de las calificaciones recibidas
desviaciones = np.std(coevaluaciones, axis=1, ddof=1)
print(desviaciones)

desviaciones_normalizadas = (autoevaluaciones - promedios) / desviaciones

[16.66666667 13.33333333 15.33333333 19.        ]
[2.081666   3.05505046 3.05505046 1.        ]


## Paso 3: Establecer el factor multiplicativo
Para determinar el factor multiplicativo $F$, podemos usar la relación entre la autoevaluación $A_i$ y la calificación promedio $prom(C_i)$. Por ejemplo, si $A_i$ está significativamente por encima de $prom(C_i)$, se penaliza, y si está significativamente por debajo, se premia.

Podemos definir $F$ como sigue:

$$
F_i = 1 + k \times \frac{A_i - prom(C_i)}{\sigma_i}
$$

donde $k$ es una constante de ajuste para controlar la magnitud del factor. Queremos que $F_i$ esté entre 0.8 y 1.2, por lo que debemos escalar y limitar este valor. 

$$
F_1 = 1 + k \times \frac{18 - 17.5}{1.5} = 1 + k \times \frac{0.5}{1.5} = 1 + \frac{k}{3} \\
F_2 = 1 + k \times \frac{15 - 15}{0.5} = 1 \\
F_3 = 1 + k \times \frac{16 - 17}{1} = 1 - k \\
$$

+ Factor del estudiante 1: $F_1 = 1 + \frac{k}{3}$
+ Factor del estudiante 2: $F_2 = 1$ 
+ Factor del estudiante 3: $F_3 = 1 - k$ 

In [34]:
# Constante de ajuste
k = 0.1

# Calcular el factor multiplicativo para cada estudiante

factores = 1 + k * desviaciones_normalizadas
print(factores)

# Ajustar los factores para que el promedio sea 1
factores = factores / np.mean(factores)

# Limitar el factor multiplicativo entre 0.8 y 1.2
factores = np.clip(factores, 0.8, 1.2)

print("Autoevaluaciones:", autoevaluaciones)
print("Promedios:", promedios)
print("Desviaciones:", desviaciones)
print("Factores multiplicativos:", factores)


Autoevaluaciones: [20 20 20 20]
Promedios: [16.66666667 13.33333333 15.33333333 19.        ]
Desviaciones: [2.081666   3.05505046 3.05505046 1.        ]
Factores multiplicativos: [1.00203279 1.0522064  0.99566227 0.95009854]
